In [1]:
import requests
from requests import get
import datetime

g_ip = 0    # Global Variable of IP Address
g_lat = 0   # Global Variable of Latitude
g_lon = 0   # Global Variable of Longnitude

## Function for collecting current IP address of current device and find Lattitude & Longitude 

In [2]:
def ip_finder():
    
    global g_ip  # Global Variable of IP Address
    global g_lat # Global Variable of Latitude
    global g_lon # Global Variable of Longnitude
    
    if(g_ip==0):
        ip = get('https://api.ipify.org')  # Fetching Current Device Public IP address
        if ip.status_code != 200:
            raise ApiError('GET /tasks/ {}'.format(ip.status_code))
        else:
            ip = ip.text
            g_ip = ip

    yourapi = 'put-your-Api-key' # Your API key
    ip_url = 'http://api.ipstack.com/'+g_ip+'?access_key='+yourapi
    jsn_ip = requests.get(ip_url)
    if jsn_ip.status_code != 200:
        raise ApiError('GET /tasks/ {}'.format(jsn_ip.status_code))
    else:
        ip_result = jsn_ip.json()
        g_lat = ip_result['latitude']      # Set the Latitude
        g_lon = ip_result['longitude']     # Set the Longitude
        

## Weather JSON collector using IP or Lat & Lon 

In [3]:
def weather_json(mode,arr):
    
    global g_ip  # Global Variable of IP Address
    global g_lat # Global Variable of Latitude
    global g_lon # Global Variable of Longnitude
    
    if (mode=='ip' or mode=='IP'):
        if(arr==0):
            if(g_ip==0):
                ip_finder()  # For Collecting IP,Lat,Lon
            else:
                g_ip = arr
                ip_finder()  # For Collecing the Lat Lon
        else:
                g_ip = arr
                ip_finder()  # For Collecing the Lat Lon
        
    elif (mode=='LATLON' or mode=='latlon'):
        if(arr[0]!=0 and arr[1]!=0  ):
            g_lat = arr[0]
            g_lon = arr[1]
        else:
            g_ip = 0
            g_lat = 0
            g_lon = 0
            ip_finder()
    else:
        g_lat = 0
        g_lon = 0
        ip_finder()
        

    
    if(g_lat==0 or g_lon==0):
        return('Error Input')
    else:
        your_darksky_api='put-you-API-key'
        url = 'https://api.darksky.net/forecast/'+your_darksky_api+'/'+str(g_lat)+','+str(g_lon)
        jsn = requests.get(url)
        if jsn.status_code != 200:
            raise ApiError('GET /tasks/ {}'.format(jsn.status_code))
        else:
            result = jsn.json()
            return(result)

## Current Weather Function :

In [4]:
def current(mod,arr):
    result = weather_json(mod,arr)
    
    dic={} # Result Dictionary
    fer = result['currently']['temperature']
    cel = (fer-32)*(5/9) # Fahrenheit to Celsius conversion 
    hum = int(result['currently']['humidity'] * 100) # Humidity on percentage
    dic["Overall Weather "]=result['currently']['summary']
    dic["Current Temperature in C"]="%.2f" % cel
    dic["Current Temperature in F"]="%.2f" %fer
    dic["Current Humidity"]=hum
    dic["Current Wind Speed"]=result['currently']['windSpeed']
    dic["Current Wind Pressure"]=result['currently']['pressure']
    return(dic)

    
    
#Parameters and return values. You can use all of this parameters to fetch the results : (Add into above)
        #time -> 1555648895        use as ->   dic['time'] = result['currently']['time']
        #summary -> "Clear"
        #icon -> "clear-day"
        #precipIntensity -> 0
        #precipProbability -> 0
        #temperature -> 95.14
        #apparentTemperature -> 95.14
        #dewPoint -> 58.83
        #humidity -> 0.3
        #pressure -> 1008.26
        #windSpeed -> 3.41
        #windGust -> 10.73
        #windBearing -> 299
        #cloudCover -> 0
        #uvIndex -> 10
        #visibility -> 10
        #ozone -> 275.36
    
    

## Hourly Forecasting 

In [5]:
def hourly(mod,arr):
    result = weather_json(mod,arr)
    dic={'Today':{"%.2f"%i:{} for i in range(25)},'Tomorrow':{"%.2f"%i:{} for i in range(25)}}
    for i in range (1,49):

        fer = result['hourly']['data'][i]['temperature']
        cel = (fer-32)*(5/9)                 # Fahrenheit to Celsius conversion 
        hum = int(result['hourly']['data'][i]['humidity'] * 100)  # Humidity on percentage

        if(i<=24):
            dic['Today']["%.2f" %i]["Overall Weather"]= result['hourly']['data'][i]['summary']
            dic['Today']["%.2f" %i]["Temperature in C"]="%.2f" % cel
            dic['Today']["%.2f" %i]["Temperature in F"]="%.2f" %fer
            dic['Today']["%.2f" %i]["Humidity"]=hum
            dic['Today']["%.2f" %i]["Wind Speed"]=result['hourly']['data'][i]['windSpeed']
            dic['Today']["%.2f" %i]["Wind Pressure"]=result['hourly']['data'][i]['pressure']

        else:
            tm  = i - 24
            dic['Tomorrow']["%.2f" %tm]["Overall Weather"]= result['hourly']['data'][i]['summary']
            dic['Tomorrow']["%.2f" %tm]["Temperature in C"]="%.2f" % cel
            dic['Tomorrow']["%.2f" %tm]["Temperature in F"]="%.2f" %fer
            dic['Tomorrow']["%.2f" %tm]["Humidity"]=hum
            dic['Tomorrow']["%.2f" %tm]["Wind Speed"]=result['hourly']['data'][i]['windSpeed']
            dic['Tomorrow']["%.2f"%tm]["Wind Pressure"]=result['hourly']['data'][i]['pressure']
        
        
    return(dic)
        
#Parameters and return values. You can use all of this parameters to fetch the results : (Add into above)
        
      
        #time -> 1555651800     to use => dic['Today']["%.2f" %tm]["time"] = result['hourly']['data'][i]['time']
        #summary ->  "Clear"
        #icon ->  "clear-day"
        #precipIntensity ->  0
        #precipProbability ->  0
        #temperature ->  96.92
        #apparentTemperature ->  96.92
        #dewPoint ->  56.92
        #humidity ->  0.27
        #pressure  -> 1007.89
        #windSpeed ->  6.61
        #windGust ->  11.28
        #windBearing ->  330
        #cloudCover  -> 0
        #uvIndex  ->  12
        #visibility ->  10
        #ozone  -> 275.35
        
    

## Weekly Forecasting 

In [6]:
def weekly(mod,arr):
    result = weather_json(mod,arr)
    x = datetime.datetime.now()
    dic = {}
    for i in range (0,7):
        r_date = x+datetime.timedelta(i)
        dic[r_date.strftime("%d-%B-%Y")]={}

    for i in range (0,7):
        r_date = x+datetime.timedelta(i)

        mfer = result['daily']['data'][i]['temperatureMax']
        mcel = (mfer-32)*(5/9)                # Fahrenheit to Celsius conversion

        Mfer = result['daily']['data'][i]['temperatureMin']
        Mcel = (Mfer-32)*(5/9)              # Fahrenheit to Celsius conversion
        hum = int(result['daily']['data'][i]['humidity'] * 100) # Humidity on percentage

        dic[r_date.strftime("%d-%B-%Y")]["Overall Weather"]=result['daily']['data'][i]['summary']
        dic[r_date.strftime("%d-%B-%Y")]["Max Temperature in C"]="%.2f" % Mcel
        dic[r_date.strftime("%d-%B-%Y")]["Max Temperature in F"]="%.2f" %Mfer
        dic[r_date.strftime("%d-%B-%Y")]["Min Temperature in C"]= "%.2f" % mcel
        dic[r_date.strftime("%d-%B-%Y")]["Min Temperature in F"]="%.2f" %mfer
        dic[r_date.strftime("%d-%B-%Y")]["Humidity"]=hum
        dic[r_date.strftime("%d-%B-%Y")]["Wind Speed"]=result['daily']['data'][i]['windSpeed']
        dic[r_date.strftime("%d-%B-%Y")]["Wind Pressure"]=result['daily']['data'][i]['pressure']
        
    return(dic)

#Parameters and return values. You can use all of this parameters to fetch the results : (Add into above)
        
        #time -> 1555612200  to use => dic[r_date.strftime("%d-%B-%Y")]["time"]=result['daily']['data'][i]['time']
        #summary -> "Partly cloudy starting in the afternoon."
        #icon -> "partly-cloudy-night"
        #sunriseTime -> 1555630991
        #sunsetTime -> 1555677088
        #moonPhase -> 0.51
        #precipIntensity -> 0.0002
        #precipIntensityMax -> 0.0012
        #precipIntensityMaxTime -> 1555662600
        #precipProbability -> 0.04
        #precipType -> "rain"
        #temperatureHigh -> 97.81
        #temperatureHighTime -> 1555655400
        #temperatureLow -> 76.66
        #temperatureLowTime -> 1555716600
        #apparentTemperatureHigh -> 97.81
        #apparentTemperatureHighTime -> 1555655400
        #apparentTemperatureLow -> 76.66
        #apparentTemperatureLowTime -> 1555716600
        #dewPoint -> 59.24
        #humidity -> 0.4
        #pressure -> 1006.44
        #windSpeed -> 4.29
        #windGust -> 26.3
        #windGustTime -> 1555695000
        #windBearing -> 248
        #cloudCover -> 0.17
        #uvIndex -> 12
        #uvIndexTime -> 1555651800
        #visibility -> 10
        #ozone -> 273.2
        #temperatureMin -> 76.3
        #temperatureMinTime -> 1555626600
        #temperatureMax -> 97.81
        #temperatureMaxTime -> 1555655400
        #apparentTemperatureMin -> 76.57
        #apparentTemperatureMinTime -> 1555626600
        #apparentTemperatureMax -> 97.81
        #apparentTemperatureMaxTime -> 1555655400

In [7]:
current('latlon',[0,0])

{'Overall Weather ': 'Mostly Cloudy',
 'Current Temperature in C': '32.34',
 'Current Temperature in F': '90.21',
 'Current Humidity': 47,
 'Current Wind Speed': 5.13,
 'Current Wind Pressure': 1006.14}

In [8]:
current('latlon',[52.3824,4.8995])

{'Overall Weather ': 'Clear',
 'Current Temperature in C': '20.25',
 'Current Temperature in F': '68.45',
 'Current Humidity': 47,
 'Current Wind Speed': 10.95,
 'Current Wind Pressure': 1029.75}

In [9]:
current('IP','Provided ip')

{'Overall Weather ': 'Mostly Cloudy',
 'Current Temperature in C': '32.33',
 'Current Temperature in F': '90.20',
 'Current Humidity': 47,
 'Current Wind Speed': 5.1,
 'Current Wind Pressure': 1006.14}

In [10]:
current('ip',0)

{'Overall Weather ': 'Mostly Cloudy',
 'Current Temperature in C': '32.33',
 'Current Temperature in F': '90.20',
 'Current Humidity': 47,
 'Current Wind Speed': 5.09,
 'Current Wind Pressure': 1006.14}

In [11]:
current(0,0)['Current Humidity']

47

In [13]:
weekly(0,0)

{'19-April-2019': {'Overall Weather': 'Humid throughout the day and foggy in the morning.',
  'Max Temperature in C': '26.16',
  'Max Temperature in F': '79.09',
  'Min Temperature in C': '32.64',
  'Min Temperature in F': '90.75',
  'Humidity': 57,
  'Wind Speed': 4.79,
  'Wind Pressure': 1008.21},
 '20-April-2019': {'Overall Weather': 'Mostly cloudy in the morning and humid throughout the day.',
  'Max Temperature in C': '24.41',
  'Max Temperature in F': '75.93',
  'Min Temperature in C': '36.37',
  'Min Temperature in F': '97.46',
  'Humidity': 51,
  'Wind Speed': 3.36,
  'Wind Pressure': 1006.67},
 '21-April-2019': {'Overall Weather': 'Mostly cloudy starting in the afternoon.',
  'Max Temperature in C': '24.99',
  'Max Temperature in F': '76.99',
  'Min Temperature in C': '36.91',
  'Min Temperature in F': '98.43',
  'Humidity': 55,
  'Wind Speed': 6.64,
  'Wind Pressure': 1005.36},
 '22-April-2019': {'Overall Weather': 'Mostly cloudy throughout the day.',
  'Max Temperature in C'

In [14]:
weekly('latlon',[52.3824,4.8995])

{'19-April-2019': {'Overall Weather': 'Partly cloudy in the morning.',
  'Max Temperature in C': '10.84',
  'Max Temperature in F': '51.51',
  'Min Temperature in C': '22.91',
  'Min Temperature in F': '73.24',
  'Humidity': 50,
  'Wind Speed': 9.97,
  'Wind Pressure': 1029.03},
 '20-April-2019': {'Overall Weather': 'Clear throughout the day.',
  'Max Temperature in C': '10.35',
  'Max Temperature in F': '50.63',
  'Min Temperature in C': '22.97',
  'Min Temperature in F': '73.34',
  'Humidity': 51,
  'Wind Speed': 9.03,
  'Wind Pressure': 1030.47},
 '21-April-2019': {'Overall Weather': 'Clear throughout the day.',
  'Max Temperature in C': '8.47',
  'Max Temperature in F': '47.24',
  'Min Temperature in C': '21.14',
  'Min Temperature in F': '70.06',
  'Humidity': 47,
  'Wind Speed': 7.11,
  'Wind Pressure': 1024.11},
 '22-April-2019': {'Overall Weather': 'Clear throughout the day.',
  'Max Temperature in C': '8.28',
  'Max Temperature in F': '46.90',
  'Min Temperature in C': '21.51'

In [16]:
hourly(0,0)

{'Today': {'0.00': {},
  '1.00': {'Overall Weather': 'Mostly Cloudy',
   'Temperature in C': '31.65',
   'Temperature in F': '88.97',
   'Humidity': 47,
   'Wind Speed': 1.4,
   'Wind Pressure': 1006.4},
  '2.00': {'Overall Weather': 'Partly Cloudy',
   'Temperature in C': '30.25',
   'Temperature in F': '86.45',
   'Humidity': 49,
   'Wind Speed': 3.43,
   'Wind Pressure': 1007.29},
  '3.00': {'Overall Weather': 'Clear',
   'Temperature in C': '28.73',
   'Temperature in F': '83.72',
   'Humidity': 51,
   'Wind Speed': 3.14,
   'Wind Pressure': 1008.25},
  '4.00': {'Overall Weather': 'Clear',
   'Temperature in C': '27.64',
   'Temperature in F': '81.75',
   'Humidity': 54,
   'Wind Speed': 8.95,
   'Wind Pressure': 1008.84},
  '5.00': {'Overall Weather': 'Clear',
   'Temperature in C': '27.02',
   'Temperature in F': '80.63',
   'Humidity': 55,
   'Wind Speed': 9.89,
   'Wind Pressure': 1009.07},
  '6.00': {'Overall Weather': 'Partly Cloudy',
   'Temperature in C': '26.59',
   'Tempe

In [18]:
hourly(0,0)['Today']['6.00']

{'Overall Weather': 'Partly Cloudy',
 'Temperature in C': '26.59',
 'Temperature in F': '79.86',
 'Humidity': 56,
 'Wind Speed': 11.26,
 'Wind Pressure': 1009.03}